In [2]:
# import getpass
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
#os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

#os.environ["YDC_API_KEY"] = getpass.getpass()
#print(os.environ)



#client = ChatGroq(
#        api_key=os.environ.get("GROQ_API_KEY"),
#    )

model = ChatGroq(model="llama-3.1-8b-instant")




In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_community.tools.you import YouSearchTool
# from langchain_community.utilities.you import YouSearchAPIWrapper

#api_wrapper = YouSearchAPIWrapper(num_web_results=1)
#search = YouSearchTool(api_wrapper=api_wrapper)


search = TavilySearchResults(max_results=2)
tools = [search]
# search.invoke({"query": "What happened at the last wimbledon"})

In [4]:
import datetime

from langchain_core.prompts import ChatPromptTemplate

today = datetime.datetime.today().strftime("%D")
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{messages}"),
    ]
)
# def modify_messages(messages: list):
#     return chat_prompt_template.invoke({"messages": messages})


In [5]:
# def print_stream(stream):
#     for s in stream:
#         message = s["messages"][-1]
#         if isinstance(message, tuple):
#             print(message)
#         else:
#             message.pretty_print()
def print_stream(stream):
    for s in stream:
        # Debug print to see what's actually in the stream
        # print("Stream item:", s)
        
        # Check if messages exist
        if 'messages' not in s or len(s['messages']) == 0:
            print("No messages in stream item")
            continue
        
        message = s['messages'][-1]
        
        # More detailed type checking and printing
        if isinstance(message, tuple):
            print("Tuple message:", message)
        elif message is None:
            print("Null message encountered")
        else:
            try:
                message.pretty_print()
            except Exception as e:
                print(f"Error printing message: {e}")
                print("Message details:", message)

In [6]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [7]:


from langgraph.prebuilt import create_react_agent

# def initialize_agent_executor():
agent_executor = create_react_agent(model, tools, checkpointer=memory)
    # return agent_executor

def run_agent_conversation():
   
    # agent_executor = initialize_agent_executor()
    
  
    config = {"configurable": {"thread_id": "14"}}

    # First input
    first_inputs = {
        "messages": [
            ("user", "Hey, I am bob and I live in PUNE. Is it a good place to enjoy my holidays?")
        ]
    }

    print("First Question:")
    print_stream(agent_executor.stream(first_inputs, config=config, stream_mode="values"))

    print("\nSecond Question:")
    # Second input
    second_inputs = {
        "messages": [
            ("user", "Tell me my name and my place name and about the weather in the place where I live. Please give the data")
        ]
    }
    
    print_stream(agent_executor.stream(second_inputs, config=config, stream_mode="values"))


# run_agent_conversation()

The below cell uses the system_message prompt from another file named system_message.py

In [8]:

# from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer
# from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor
from system_message import system_message_function
system_message = system_message_function()
langgraph_agent_executor  = create_react_agent(
       model, tools, state_modifier=system_message, checkpointer=memory
   )
# langgraph_agent_executor = AgentExecutor(agent=agent, tools=tools)

def langgraph():
    
   
 
   
# with open('system_message.txt', 'r',encoding = 'utf-8') as file:
#     system_message_content = file.read()
# system_message = "You are a helpful assistant."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

# memory = MemorySaver()
  

   config = {"configurable": {"thread_id": "test-thread"}}
   print(
    langgraph_agent_executor.invoke(
        {
            "messages": [
                ("user", "Hi, I'm polly! What are the property details avaialable in pune. I want 3 BHK ")
            ]
        },
        config,
    )["messages"][-1]
   )
   print("---")
   print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "Remember my name?")]}, config
    )["messages"][-1]
   )
   print("---")
   print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "what was that output again?")]}, config
    )["messages"][-1]
   )
# langgraph()

Still need to work on the below code

In [9]:


# async for event in agent_executor.astream_events(
#     {"messages": [HumanMessage(content="whats the weather in pune?")]}, version="v1"
# ):
#     kind = event["event"]
#     if kind == "on_chain_start":
#         if (
#             event["name"] == "Agent"
#         ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
#             print(
#                 f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
#             )
#     elif kind == "on_chain_end":
#         if (
#             event["name"] == "Agent"
#         ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
#             print()
#             print("--")
#             print(
#                 f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
#             )
#     if kind == "on_chat_model_stream":
#         content = event["data"]["chunk"].content
#         if content:
#             # Empty content in the context of OpenAI means
#             # that the model is asking for a tool to be invoked.
#             # So we only print non-empty content
#             print(content, end="|")
#     elif kind == "on_tool_start":
#         print("--")
#         print(
#             f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
#         )
#     elif kind == "on_tool_end":
#         print(f"Done tool: {event['name']}")
#         print(f"Tool output was: {event['data'].get('output')}")
#         print("--")
